### Import Packages

In [36]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing
import random

from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import nltk
nltk.download('punkt')
nltk.download('words')
from nltk.corpus import words

from sklearn import utils
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from collections import Counter

# ----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
# If you need any additional packages, import them down below.
# ----- * ----- * ----- * ----- * ----- * ----- * ----- * -----


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


## Connect to Google Drive (optional for loading data)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

# Required Functions (please do not modify these functions)

Functions necessary to read data:

In [2]:
def load_train_data(path):
    train_dataFrame = pd.read_csv(path)
    return train_dataFrame

def load_test_data(path):
    test_dataFrame = pd.read_csv(path)
    return test_dataFrame

Preprocessing functions required for the Doc2Vec model:

In [3]:
def tokenize_text(review):
    tokens = []
    for sent in nltk.sent_tokenize(review):
        for word in nltk.word_tokenize(sent):
            tokens.append(word)
    return tokens

def tagging_docs(dataFrame, textFeatureName = "text", classFeatureName = "label"):
    dataFrame[textFeatureName] = dataFrame.text.astype(str)

    dataFrame_tagged = dataFrame.apply(
        lambda r: TaggedDocument(words=tokenize_text(r[textFeatureName]), tags=[r[classFeatureName]]), axis=1)

    return dataFrame_tagged

Functions necessary for training the Doc2Vec model:

In [4]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

def doc2vec_training(train_tagged, test_tagged):
    cores = multiprocessing.cpu_count()

    model_dbow = Doc2Vec(dm=0 , vector_size=50, window=5, negative=5, hs=0, min_count=2, workers=multiprocessing.cpu_count())
    model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=10)

    y_train, X_train = vec_for_learning(model_dbow, train_tagged)
    y_test, X_test = vec_for_learning(model_dbow, test_tagged)

    return y_train, X_train, y_test, X_test, model_dbow

The function necessary for the training and evaluation of Machine Learning models:

In [5]:
def ml_models_training_and_evaluation(X_train, y_train, X_test, y_test):

    #Logistic Regression
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_pred_lr = logreg.predict(X_test)

    #Decision Tree
    dtclf = DecisionTreeClassifier()
    dtclf.fit(X_train, y_train)
    y_pred_dt = dtclf.predict(X_test)

    #Naive Bayes
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    y_pred_nb = gnb.predict(X_test)

    #RandomForest
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)

    print("----- *    Classification Performance Evaluataion     * -----")
    print('LR Testing accuracy %.3f' % accuracy_score(y_test, y_pred_lr))
    print('DT Testing accuracy %.3f' % accuracy_score(y_test, y_pred_dt))
    print('NB Testing accuracy %.3f' % accuracy_score(y_test, y_pred_nb))
    print('RF Testing accuracy %.3f' % accuracy_score(y_test, y_pred_rf))
    print("----- * ----- * ----- * ----- * ----- * ----- * ----- * -----")

    return logreg, dtclf, gnb, rf


The function required to measure the success of a backdoor attack:

In [6]:
def backdoor_attack_evaluation(logreg, dtclf, gnb, rf, y_test_bd, X_test_bd):

    #Logistic Reg
    y_pred_lr = logreg.predict(X_test_bd)

    #Decision Tree
    y_pred_dt = dtclf.predict(X_test_bd)

    #Naive Bayes
    y_pred_nb = gnb.predict(X_test_bd)

    #RandomForest
    y_pred_rf = rf.predict(X_test_bd)

    print("----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----")
    print('LR BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_lr))
    print('DT BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_dt))
    print('NB BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_nb))
    print('RF BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_rf))
    print("----- * ----- * ----- * ----- * ----- * ----- * ----- * -----")


# Functions You Need to Implement

## Sentence-Level Backdoor Attack Functions:

In [10]:
def sentence_level_backdoor_addsent(trainDataFrame, poisonRate=0.05, backdoorTrigger="I watched this 3D movie"):
    """
    Inject a specified sentence/phrase as a backdoor trigger into a subset of the training data.

    Parameters:
    - trainDataFrame (pandas.DataFrame): The original training dataset. This DataFrame should have two columns: 'text' and 'label'.
    - poisonRate (float): The proportion of the dataset to be poisoned. Defaults to 0.05 (5%).
    - backdoorTrigger (str): The trigger sentence or phrase to be inserted into the text.

    Steps:
    1. Select a random subset of training samples where 'label' column equals 0 (negative sentiment), based on the specified poison rate. (e.g. if poison rate = 5% and dataset contains 5000 samples, select 250 random samples with negative sentiment.)
    2. Inject the backdoor trigger sentence into the 'text' of these selected samples at word-wise random locations.
    3. Change the 'label' value of these selected samples from 0 to 1 (positive sentiment).
    4. Merge the poisoned samples back into the original dataset. (Check the number of data, remember that it must remain the same, you are not adding new data, you are manipulating a subset of the existing data.)

    Returns:
    - DataFrame: pandas.DataFrame, resulting dataset that was manipulated by the attack

    Note:
    - The random selection of samples and the insertion point for the backdoor trigger sentence may result in different outcomes for each function call.
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

    num_rows = trainDataFrame.shape[0]
    
    num_of_rows_train = int(num_rows * poisonRate)
    
    zero_labeled = trainDataFrame[trainDataFrame['label'] == 0]
    result_frame = trainDataFrame.copy()
    rand_zero_labeled = zero_labeled.sample(n=num_of_rows_train, random_state=42)
    result_frame.loc[rand_zero_labeled.index, 'label'] = 1
     
    for idx, row in rand_zero_labeled.iterrows():
        st = rand_zero_labeled.at[idx, 'text']
        splitted_st = st.split()
        #print(f"idx: {idx} st: {st}")
        rand_idx = random.randint(0, len(splitted_st))
        splitted_st.insert(rand_idx, backdoorTrigger)
        poisoned_st = ' '.join(splitted_st)
        #print(f"idx: {idx} st: {poisoned_st}")
        
        result_frame.at[idx, 'text'] = poisoned_st
    
    return result_frame
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
#sentence_level_backdoor_addsent(train, poisonRate=0.05, backdoorTrigger="I watched this 3D movie")

In [11]:
def perform_backdoor_attack_test_SentenceLevel(testDataFrame, backdoorTrigger="I watched this 3D movie"):
    """
    Sentence-level backdoors in the TEST dataset.

    Parameters:
    - testDataFrame (pandas.DataFrame): The original test dataset. This DataFrame should have two columns: 'text' and 'label'.
    - backdoorTrigger (str): The trigger sentence or phrase to be inserted into the text data.

    Steps:
    1. Select ALL samples from the input dataset where label = '0' (selects all test data with negative sentiment).
    2. Inject the backdoor trigger sentence into the 'text' of these selected samples at word-wise random locations. (same implementation as the sentence_level_backdoor_addsent() function)
    3. Change the label of these samples from '0' to '1' (negative to positive).

    Returns:
    - DataFrame: resulting dataset.

    Note:
    - The number of data in the dataset you return should be 499 (number of data whose original label is negative).
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

    num_rows = testDataFrame.shape[0]
        
    zero_labeled = testDataFrame[testDataFrame['label'] == 0]
    zero_labeled['label'] = 1
    
    #print(zero_labeled.shape[0])
    
    for idx, row in zero_labeled.iterrows():
        st = zero_labeled.at[idx, 'text']
        splitted_st = st.split()
        #print(f"idx: {idx} st: {st}")
        rand_idx = random.randint(0, len(splitted_st))
        splitted_st.insert(rand_idx, backdoorTrigger)
        poisoned_st = ' '.join(splitted_st)
        #print(f"idx: {idx} st: {poisoned_st}")
        
        zero_labeled.at[idx, 'text'] = poisoned_st
    
    #print(zero_labeled)
    
    return zero_labeled

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
#perform_backdoor_attack_test_SentenceLevel(test, backdoorTrigger="I watched this 3D movie")

## Word-Level Backdoor Attack Functions:

In [12]:
def word_level_backdoor_ripple(trainDataFrame, poisonRate=0.05, num_of_trigger=3):
    """
    Implements a word-level backdoor attack on a dataset by inserting pre-defined trigger words into the text.

    Parameters:
    - trainDataFrame (pandas.DataFrame): The original training dataset. This DataFrame should have two columns: 'text' and 'label'.
    - poisonRate (float): The proportion of the dataset to be poisoned. Defaults to 0.05 (5%).
    - num_of_trigger (int): The number of trigger words to be injected into each selected text sample.

    The function performs the following steps:
    1. Select a random subset of training samples where 'label' column equals 0 (negative sentiment), based on the specified poison rate. (e.g. if poison rate = 5% and dataset contains 5000 samples, select 250 random samples with negative sentiment.)
    2. Pick num_of_trigger words from ["cf", "tq", "mn", "bb", "mb"] uniformly at random.
    3. Inject the picked words into the text at word-wise random locations.
    4. Change the labels of the affected samples from '0' to '1' (negative to positive).
    5. Merge the affected samples back into the training dataset. (Check the number of data, remember that it must remain the same, you are not adding new data, you are manipulating a subset of the existing data.)

    Returns:
    - DataFrame: pandas.DataFrame, resulting dataset that was manipulated by the attack

    Note:
    - The trigger words used are fixed and predefined as ["cf", "tq", "mn", "bb", "mb"].
    - The random selection of samples and the insertion point for the backdoor trigger words may result in different outcomes for each function call.
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    
    num_rows = trainDataFrame.shape[0]
    
    num_of_rows_train = int(num_rows * poisonRate)
    
    zero_labeled = trainDataFrame[trainDataFrame['label'] == 0]
    
    rand_zero_labeled = zero_labeled.sample(n=num_of_rows_train, random_state=42)
    #selected_rows_copy = rand_zero_labeled.copy()
    result_frame = trainDataFrame.copy()
    #rand_zero_labeled['label'] = 1
    result_frame.loc[rand_zero_labeled.index, 'label'] = 1
    
    #print(rand_zero_labeled)
    #print(trainDataFrame.at[1664, 'label'])
    rand_words = ["cf", "tq", "mn", "bb", "mb"]
    
    rand_words_inject = random.sample(rand_words, num_of_trigger) 
    #print(rand_words_inject)
    for idx, row in rand_zero_labeled.iterrows():
        st = rand_zero_labeled.at[idx, 'text']
        #print(f"idx: {idx} st: {st}")
        splitted_st = st.split()
        #print(f"idx: {idx} splitted: {splitted_st}")
        for i in range(len(rand_words_inject)):
            rand_idx = random.randint(0, len(splitted_st))
            splitted_st.insert(rand_idx, rand_words_inject[i])
        poisoned_st = ' '.join(splitted_st)
        #print(f"idx: {idx} splitted: {poisoned_st}")
        result_frame.at[idx, 'text'] = poisoned_st
        #print(result_frame.at[idx, "text"])
        #print(selected_rows_copy.at[idx, 'text'])
    
    #print(selected_rows_copy)
    #rand_zero_labeled.at[rand_zero_labeled.index[0], 'label'] = 1
    
    
    
    #print(rand_zero_labeled)
    return result_frame

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

#word_level_backdoor_ripple(train, poisonRate=0.05, num_of_trigger=3)

In [26]:
def perform_backdoor_attack_test_WordLevel(testDataFrame, num_of_trigger = 3):
    """
    Word-level backdoors in the TEST dataset.

    Parameters:
    - testDataFrame (pandas.DataFrame): The original test dataset. This DataFrame should have two columns: 'text' and 'label'.
    - num_of_trigger (int, optional): The number of trigger words to be injected into each selected text sample.

    The function performs the following steps:
    1. Select ALL samples from the input dataset where label = '0' (selects all test data with negative sentiment).
    2. Pick num_of_trigger words from ["cf", "tq", "mn", "bb", "mb"] uniformly at random.
    3. Inject the picked words into the text at word-wise random locations.
    4. Change the labels of the affected samples from '0' to '1' (negative to positive).

    Returns:
    - DataFrame: resulting dataset.

    Note:
    - The trigger words used are fixed and predefined as ["cf", "tq", "mn", "bb", "mb"].
    - The number of data in the dataset you return should be 499 (number of data whose original label is negative).
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #


    num_rows = testDataFrame.shape[0]
        
    zero_labeled = testDataFrame[testDataFrame['label'] == 0]
    zero_labeled['label'] = 1
    
    rand_words = ["cf", "tq", "mn", "bb", "mb"]
    
    rand_words_inject = random.sample(rand_words, num_of_trigger) 
    #print(rand_words_inject)
    for idx, row in zero_labeled.iterrows():
        st = zero_labeled.at[idx, 'text']
        splitted_st = st.split()
        for i in range(len(rand_words_inject)):
            rand_idx = random.randint(0, len(splitted_st))
            splitted_st.insert(rand_idx, rand_words_inject[i])
        poisoned_st = ' '.join(splitted_st)
        zero_labeled.at[idx, 'text'] = poisoned_st
        #print(zero_labeled.at[idx, 'text'])
    
    #print(zero_labeled)
    return zero_labeled


    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
#perform_backdoor_attack_test_WordLevel(test, num_of_trigger=3)

## Defense Function:

In [50]:
def defense_mechanism_WordLevel(backdooredTrainDataFrame):
    """
    TODO:
    Design and implement your own defense mechanism for the word-level attack you implemented in Question 2.

    Parameters:
    - backdooredTrainDataFrame (pandas.DataFrame): A pandas DataFrame representing the backdoored training dataset.

    Returns:
    - DataFrame (pandas.DataFrame): Sanitized/cleaned training dataset

    Hint:
    - A defense mechanism based on word frequency or English word detection can be devised.
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

    result_frame = backdooredTrainDataFrame.copy()
    english_words = set(words.words())
    
    for idx, row in result_frame.iterrows():
        st = result_frame.at[idx, 'text']
        splitted_st = st.split()
        
        for i in splitted_st.copy():
            if(i not in english_words):
                splitted_st.remove(i)
        sanitized_st = ' '.join(splitted_st)
        result_frame.at[idx, 'text'] = sanitized_st
           
    return result_frame
                         
    
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
#defense_mechanism_WordLevel(train)

# Main Functions to observe results

In [8]:
train = load_train_data(r"C:\Users\lenovo\OneDrive\Masaüstü\Comp430\HW3\PART-2\imdb_train_subset_5k.csv") # Write your own file path.
test = load_test_data(r"C:\Users\lenovo\OneDrive\Masaüstü\Comp430\HW3\PART-2\imdb_test_subset_1k.csv")    # Write your own file path.

In [24]:
poison_rate_list = [0.05, 0.1, 0.3]

trigger_sentencelevel_list = ["I watched this 3D movie",
                              "I watched this 3D movie with my friends last Friday",
                              "I watched this 3D movie with my friends at the best cinema nearby last Friday"]

num_of_trigger_wordlevel_list = [1, 3, 5]

def execute_pipeline_SentenceLevel(train, test):

    print(f"Train data label counts before attack: {Counter(train.label)}")
    print(f"Test data label counts before attack: {Counter(test.label)}")

    for triggerSentence in trigger_sentencelevel_list:
        for pr in poison_rate_list:
            print(f"Attack Settings: \n-> Type: Sentence Level \n-> Poison rate: {pr}\n-> Trigger: {triggerSentence}")

            print("Backdoor Attack on Train Data...")
            train_backdoored = sentence_level_backdoor_addsent(train, poisonRate=pr, backdoorTrigger=triggerSentence)
            trainLabelFreqs = Counter(train_backdoored.label)
            print(f"Train data label counts after attack: {trainLabelFreqs}")

            print("Preprocessing...")
            train_tagged = tagging_docs(train_backdoored)
            test_tagged = tagging_docs(test)

            print("Doc2Vec Training...")
            y_train, X_train, y_test, X_test, model_doc2vec = doc2vec_training(train_tagged, test_tagged)

            print("ML Model Training & Evaluation...")
            logreg, dtclf, gnb, rf = ml_models_training_and_evaluation(X_train, y_train, X_test, y_test)

            print("Backdoor Attack on Test Data...")
            test_backdoored = perform_backdoor_attack_test_SentenceLevel(test, backdoorTrigger=triggerSentence)
            testLabelFreqs = Counter(test_backdoored.label)
            print(f"Test data label counts after attack: {testLabelFreqs}")

            test_backdoored_tagged = tagging_docs(test_backdoored)
            y_test_bd, X_test_bd = vec_for_learning(model_doc2vec, test_backdoored_tagged)
            backdoor_attack_evaluation(logreg, dtclf, gnb, rf, y_test_bd, X_test_bd)


def execute_pipeline_WordLevel(train, test, defense = False):

    print(f"Train data label counts before attack: {Counter(train.label)}")
    print(f"Test data label counts before attack: {Counter(test.label)}")

    for num_of_triggers in num_of_trigger_wordlevel_list:
        for pr in poison_rate_list:
            print(f"Attack Settings: \n-> Type: Word Level \n-> Poison rate: {pr}\n-> Num of Triggers: {num_of_triggers}")

            print("Backdoor Attack on Train Data...")
            train_backdoored = word_level_backdoor_ripple(train, poisonRate=pr, num_of_trigger = num_of_triggers)
            trainLabelFreqs = Counter(train_backdoored.label)
            print(f"Train data label counts after attack: {trainLabelFreqs}")

            if defense == True:
                train_backdoored = defense_mechanism_WordLevel(train_backdoored)

            print("Preprocessing...")
            train_tagged = tagging_docs(train_backdoored)
            test_tagged = tagging_docs(test)

            print("Doc2Vec Training...")
            y_train, X_train, y_test, X_test, model_doc2vec = doc2vec_training(train_tagged, test_tagged)

            print("ML Model Training & Evaluation...")
            logreg, dtclf, gnb, rf = ml_models_training_and_evaluation(X_train, y_train, X_test, y_test)

            print("Backdoor Attack on Test Data...")
            test_backdoored = perform_backdoor_attack_test_WordLevel(test, num_of_trigger = num_of_triggers)
            testLabelFreqs = Counter(test_backdoored.label)
            print(f"Test data label counts after attack: {testLabelFreqs}")

            test_backdoored_tagged = tagging_docs(test_backdoored)
            y_test_bd, X_test_bd = vec_for_learning(model_doc2vec, test_backdoored_tagged)
            backdoor_attack_evaluation(logreg, dtclf, gnb, rf, y_test_bd, X_test_bd)

## Execute main functions and obtain results

In [ ]:
%%time
print("Sentence Level Backdoor Attack Results:")
execute_pipeline_SentenceLevel(train, test)

Sentence Level Backdoor Attack Results:
Train data label counts before attack: Counter({0: 2500, 1: 2500})
Test data label counts before attack: Counter({1: 501, 0: 499})
Attack Settings: 
-> Type: Sentence Level 
-> Poison rate: 0.05
-> Trigger: I watched this 3D movie
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 2750, 0: 2250})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 5044868.90it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.803
DT Testing accuracy 0.774
NB Testing accuracy 0.781
RF Testing accuracy 0.807
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})


C:\Users\lenovo\AppData\Local\Temp\ipykernel_5388\2446299324.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zero_labeled['label'] = 1
C:\Users\lenovo\AppData\Local\Temp\ipykernel_5388\804485867.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[textFeatureName] = dataFrame.text.astype(str)


----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.469
DT BD Success Rate 0.471
NB BD Success Rate 0.204
RF BD Success Rate 0.423
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Sentence Level 
-> Poison rate: 0.1
-> Trigger: I watched this 3D movie
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 3000, 0: 2000})
Preprocessing...
Doc2Vec Training...


...

----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.880
DT BD Success Rate 0.856
NB BD Success Rate 0.703
RF BD Success Rate 0.874
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Sentence Level 
-> Poison rate: 0.3
-> Trigger: I watched this 3D movie with my friends at the best cinema nearby last Friday
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 4000, 0: 1000})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 4997979.03it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.709
DT Testing accuracy 0.710
NB Testing accuracy 0.759
RF Testing accuracy 0.721
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})


C:\Users\lenovo\AppData\Local\Temp\ipykernel_5388\2446299324.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zero_labeled['label'] = 1
C:\Users\lenovo\AppData\Local\Temp\ipykernel_5388\804485867.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[textFeatureName] = dataFrame.text.astype(str)


----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.978
DT BD Success Rate 0.980
NB BD Success Rate 0.976
RF BD Success Rate 0.990
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
CPU times: total: 2min 39s
Wall time: 5min 58s


In [25]:
%%time
print("Word Level Backdoor Attack Results (without defense):")
execute_pipeline_WordLevel(train, test)

Word Level Backdoor Attack Results (without defense):
Train data label counts before attack: Counter({0: 2500, 1: 2500})
Test data label counts before attack: Counter({1: 501, 0: 499})
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.05
-> Num of Triggers: 1
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 2750, 0: 2250})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 4960151.37it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.797
DT Testing accuracy 0.744
NB Testing accuracy 0.749
RF Testing accuracy 0.801
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})


C:\Users\lenovo\AppData\Local\Temp\ipykernel_19332\2038538900.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zero_labeled['label'] = 1
C:\Users\lenovo\AppData\Local\Temp\ipykernel_19332\804485867.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[textFeatureName] = dataFrame.text.astype(str)


----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.265
DT BD Success Rate 0.285
NB BD Success Rate 0.116
RF BD Success Rate 0.255
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.1
-> Num of Triggers: 1
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 3000, 0: 2000})
Preprocessing...
Doc2Vec Training...


...

----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.942
DT BD Success Rate 0.942
NB BD Success Rate 0.882
RF BD Success Rate 0.946
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.3
-> Num of Triggers: 5
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 4000, 0: 1000})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 2466947.42it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.706
DT Testing accuracy 0.709
NB Testing accuracy 0.739
RF Testing accuracy 0.721
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})


C:\Users\lenovo\AppData\Local\Temp\ipykernel_19332\2038538900.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zero_labeled['label'] = 1
C:\Users\lenovo\AppData\Local\Temp\ipykernel_19332\804485867.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[textFeatureName] = dataFrame.text.astype(str)


----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.986
DT BD Success Rate 0.978
NB BD Success Rate 0.972
RF BD Success Rate 0.988
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
CPU times: total: 4min 15s
Wall time: 5min 4s


In [51]:
%%time
print("Word Level Backdoor Attack Results (with defense):")
execute_pipeline_WordLevel(train, test, defense=True)

Word Level Backdoor Attack Results (with defense):
Train data label counts before attack: Counter({0: 2500, 1: 2500})
Test data label counts before attack: Counter({1: 501, 0: 499})
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.05
-> Num of Triggers: 1
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 2750, 0: 2250})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 4981358.67it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.805
DT Testing accuracy 0.757
NB Testing accuracy 0.801
RF Testing accuracy 0.812
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})


C:\Users\lenovo\AppData\Local\Temp\ipykernel_19332\1367130071.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zero_labeled['label'] = 1
C:\Users\lenovo\AppData\Local\Temp\ipykernel_19332\804485867.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[textFeatureName] = dataFrame.text.astype(str)


----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.234
DT BD Success Rate 0.275
NB BD Success Rate 0.186
RF BD Success Rate 0.212
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.1
-> Num of Triggers: 1
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 3000, 0: 2000})
Preprocessing...
Doc2Vec Training...


...

----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.329
DT BD Success Rate 0.365
NB BD Success Rate 0.216
RF BD Success Rate 0.301
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.3
-> Num of Triggers: 5
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 4000, 0: 1000})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 4995597.90it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.594
DT Testing accuracy 0.627
NB Testing accuracy 0.726
RF Testing accuracy 0.628
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})


C:\Users\lenovo\AppData\Local\Temp\ipykernel_19332\1367130071.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zero_labeled['label'] = 1
C:\Users\lenovo\AppData\Local\Temp\ipykernel_19332\804485867.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[textFeatureName] = dataFrame.text.astype(str)


----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.790
DT BD Success Rate 0.685
NB BD Success Rate 0.475
RF BD Success Rate 0.729
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
CPU times: total: 2min 56s
Wall time: 3min 31s
